In [3]:
import pandas as pd
# Set the float format to 2 decimal places
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.00,0.97,1.00,N,161,141,2,9.30,1.00,0.50,0.00,0.00,1.00,14.30,2.50,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.00,1.10,1.00,N,43,237,1,7.90,1.00,0.50,4.00,0.00,1.00,16.90,2.50,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.00,2.51,1.00,N,48,238,1,14.90,1.00,0.50,15.00,0.00,1.00,34.90,2.50,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.00,1.90,1.00,N,138,7,1,12.10,7.25,0.50,0.00,0.00,1.00,20.85,0.00,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.00,1.43,1.00,N,107,79,1,11.40,1.00,0.50,3.28,0.00,1.00,19.68,2.50,0.00


In [6]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [7]:
df.columns.size

19

In [8]:
df['duration_minutes'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [9]:
df.duration_minutes.describe()

count   3066766.00
mean         15.67
std          42.59
min         -29.20
25%           7.12
50%          11.52
75%          18.30
max       10029.18
Name: duration_minutes, dtype: float64

In [10]:
# Next, we need to check the distribution of the duration variable. There are some outliers. 
# Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

df_filtered = df[(df['duration_minutes'] >= 1) & (df['duration_minutes'] <= 60)]

# Fraction of the records left
df_filtered.shape[0] / df.shape[0]


0.9812202822125979

In [11]:
# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
df_filtered.loc[:, 'PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered.loc[:, 'DOLocationID'] = df_filtered['DOLocationID'].astype(str)

# Turn a dataframe into a list of dictionaries
df_dict = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit a dictionary vectorizer and get a feature matrix from it
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)
X = dv.fit_transform(df_dict)


/var/folders/_9/6kpf33ps09ggdbg51_222pxc0000gn/T/ipykernel_68513/3641381039.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_filtered.loc[:, 'PULocationID'] = df_filtered['PULocationID'].astype(str)
/var/folders/_9/6kpf33ps09ggdbg51_222pxc0000gn/T/ipykernel_68513/3641381039.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_filtered.loc[:, 'DOLocationID'] = df_filtered['DOLocationID'].astype(str)


In [12]:
# Dimensionality of the feature matrix
X.shape

(3009173, 515)

In [13]:
# Train a plain linear regression model with default parameters
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
# model.fit(X, df_filtered['duration_minutes'])
# y_pred = model.predict(X)

In [14]:
# Calculate the RMSE of the model on the training data
rmse = mean_squared_error(df_filtered['duration_minutes'], y_pred, squared=False)


NameError: name 'y_pred' is not defined